# Find car in image
Idea: iteratively crop image and apply detection.  

### Apply on single crop

In [ ]:
# imports
import keras.models as models
import numpy as np
import os
from PIL import Image

In [ ]:
# Force CPU @todo use GPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
# Get root ws path
import plai.workspace.init # source setup.bash
ws_root = plai.workspace.init.get_ws_path()

In [ ]:
# Load image
img = Image.open("../res/bild1.jpg")

# Resize and crop image
img = img.resize( (round(img.size[0]/10), round(img.size[1]/10)), resample=Image.BICUBIC)
img = img.resize((32, 32), resample=Image.BICUBIC)
img

In [ ]:
# Convert to array and scale
data = np.asarray(img).astype(np.float32)
data2 = np.asarray(img).astype(np.float32)/255. # Second detector scaled!

In [ ]:
# Load model
model = models.load_model( os.path.join(ws_root, 'res', 'trained', 'build', 'cardetector.h5'))
#model2 = models.load_model( os.path.join(ws_root, 'res', 'trained', 'build', 'cardetector2.h5'))

In [ ]:
# Predict if car in image
model.predict(data.reshape(1, 32, 32, 3))
#model2.predict(data2.reshape(1, 32, 32, 3))

### Iteratively crop and apply


In [ ]:
# Load image
scale = 10
img = Image.open("../res/bild1.jpg")
img = img.resize( (round(img.size[0]/scale), round(img.size[1]/scale)), resample=Image.BICUBIC)
img

In [ ]:
# Load model
model = models.load_model( os.path.join(ws_root, 'res', 'trained', 'build', 'cardetector.h5') )

In [ ]:
# Predict car in image by iterating through image 
import matplotlib.pyplot as plt
from PIL import ImageDraw

size = 100
step_size = 10

cars = []

for x in range(0, img.size[0] - size, step_size):
    for y in range(0, img.size[1] - size, step_size):
        
        # Crop and convert image
        img_crop = img.crop((x, y, x + size, y + size ))
        # plt.imshow(img_crop)
        # plt.show()
        data = np.asarray(img_crop.resize((32,32), resample=Image.BICUBIC))
        data = data.astype(np.float32)
        
        # Predict
        pred = model.predict(data.reshape(-1, 32, 32, 3))
        #print(pred[0][0])
        if pred[0][0] > 0.9:
            cars.append((x,y))



In [ ]:
# Draw detection rectangles
img_out = img.copy()
draw = ImageDraw.Draw(img_out)

for car in cars:
    # draw.rectangle([car, (car[0] + size, car[1] + size) ])
    draw.line( [car, (car[0], car[1] + size), (car[0] + size, car[1] + size), (car[0] + size, car[1]), car],
             "yellow", 2)
    
    draw.point( (car[0] + 50, car[1] + 50), "yellow")

img_out

In [ ]:
# @todo Cluster points (kmean) and merge these cells

### Also vary the crop size!

In [ ]:
# Load image
scale = 5
img = Image.open("../res/bild1.jpg")
img = img.resize( (round(img.size[0]/scale), round(img.size[1]/scale)), resample=Image.BICUBIC)
img

In [ ]:
# Implement car detector function
def car_detector(img, model, step_size):
    
    sizes = [100, 50, 20]
    cars = []
    for size in sizes:
        for x in range(0, img.size[0] - size, step_size):
            for y in range(0, img.size[1] - size, step_size):

                # Crop and convert image
                img_crop = img.crop((x, y, x + size, y + size ))
                # plt.imshow(img_crop)
                # plt.show()
                data = np.asarray(img_crop.resize((32,32), resample=Image.BICUBIC))
                data = data.astype(np.float32)

                # Predict
                pred = model.predict(data.reshape(-1, 32, 32, 3))
                #print(pred[0][0])
                if pred[0][0] > 0.95:
                    cars.append((x, y, size, pred))
    return cars

cars = car_detector(img, model, 20)

In [ ]:
# Draw detection rectangles
img_out = img.copy()
draw = ImageDraw.Draw(img_out)

for car in cars:
    size = car[2]
    draw.line( [car[0:2], (car[0], car[1] + size), (car[0] + size, car[1] + size), (car[0] + size, car[1]), car[0:2]],"yellow", 2)
    

img_out

### Next step: Implement YOLO
YOLO  
- https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Redmon_You_Only_Look_CVPR_2016_paper.pdf  
YOL09000
- http://openaccess.thecvf.com/content_cvpr_2017/papers/Redmon_YOLO9000_Better_Faster_CVPR_2017_paper.pdf  
YOLOv3  
- https://arxiv.org/pdf/1804.02767.pdf